In [1]:
import pandas as pd

In [2]:
dataset_source_url="https://raw.githubusercontent.com/erkansirin78/datasets/master/Advertising.csv"

In [3]:
df = pd.read_csv(dataset_source_url)

In [4]:
df.head()

,ID,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [5]:
X = df.iloc[:,1:-1].values
X.shape

(200, 3)

In [6]:
X[:3]

array([[230.1,  37.8,  69.2],
       [ 44.5,  39.3,  45.1],
       [ 17.2,  45.9,  69.3]])

In [7]:
# y (küçük y) output, bağımlı değişken, hedef değişken diye adlandırılır
# y = df['Sales'].values
y = df.iloc[:,-1].values
y.shape

(200,)

In [8]:
y[:3]

array([22.1, 10.4,  9.3])

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
            test_size=0.25, random_state = 142)

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(150, 3)
(150,)
(50, 3)
(50,)


In [11]:
alpha = 0.5
l1_ratio = 0.5

# MLflow

In [12]:
import os
os.environ['MLFLOW_TRACKING_URI'] = 'http://localhost:5001/'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://localhost:9000/'

In [13]:
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

In [14]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [22]:
mlflow.set_experiment("Iris")

2025/11/29 05:41:24 INFO mlflow.tracking.fluent: Experiment with name 'IrisV2' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/6', creation_time=1764384084555, experiment_id='6', last_update_time=1764384084555, lifecycle_stage='active', name='IrisV2', tags={}>

## Without model registry

In [20]:
 with mlflow.start_run(run_name="noreg-elasticnet-sklearn"):
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(X_train, y_train)

        predicted_qualities = lr.predict(X_test)

        (rmse, mae, r2) = eval_metrics(y_test, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        
        # Log dataset
        # Create an instance of a PandasDataset
        dataset = mlflow.data.from_pandas(df, source=dataset_source_url, name="Advertising", targets="Sales")
        mlflow.log_input(dataset, context="training")
     
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(lr, "model",  input_example=X_train[:5],  # Sample input
                                      signature=mlflow.models.infer_signature(X_train, predicted_qualities))
        else:
            mlflow.sklearn.log_model(lr, "model",  input_example=X_train[:5],  # Sample input
                                      signature=mlflow.models.infer_signature(X_train, predicted_qualities))

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 2.0508844356901177
  MAE: 1.4013546795423537
  R2: 0.8329810010483056


/home/train/vbo-mlops/uv_env/.venv/lib64/python3.12/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'https://raw.githubusercontent.com/erkansirin78/datasets/master/Advertising.csv'. Exception: 
  return _dataset_source_registry.resolve(
/home/train/vbo-mlops/uv_env/.venv/lib64/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <

🏃 View run noreg-elasticnet-sklearn at: http://localhost:5001/#/experiments/2/runs/34eefe9aaee445fc8c045cb67868b795
🧪 View experiment at: http://localhost:5001/#/experiments/2


## With model registry

In [21]:
with mlflow.start_run(run_name="withreg-elasticnet-sklearn"):
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(X_train, y_train)

        predicted_qualities = lr.predict(X_test)

        (rmse, mae, r2) = eval_metrics(y_test, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        
        # Set a tag that we can use to remind ourselves what this run was for
        mlflow.set_tag("Training Info", "Basic Elasticnet model for Advertising data")
    
        # Infer the model signature
        signature = infer_signature(X_train, lr.predict(X_train))

        # Log dataset
        # Create an instance of a PandasDataset
        dataset = mlflow.data.from_pandas(df, source=dataset_source_url, name="Advertising", targets="Sales")
        mlflow.log_input(dataset, context="training")
    
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(sk_model=lr, 
                                     artifact_path="model", 
                                     signature=signature,
                                     input_example=X_train,
                                     registered_model_name="AdvertisingElasticnetModel")
        else:
            mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 2.0508844356901177
  MAE: 1.4013546795423537
  R2: 0.8329810010483056


/home/train/vbo-mlops/uv_env/.venv/lib64/python3.12/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'https://raw.githubusercontent.com/erkansirin78/datasets/master/Advertising.csv'. Exception: 
  return _dataset_source_registry.resolve(
/home/train/vbo-mlops/uv_env/.venv/lib64/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <

🏃 View run withreg-elasticnet-sklearn at: http://localhost:5001/#/experiments/2/runs/e280c07049364400b0902e7f4172299f
🧪 View experiment at: http://localhost:5001/#/experiments/2


# Deleting Experiment Issue

In [17]:
# Option 1: Restore the deleted experiment
import mlflow
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name("Iris")
client.restore_experiment(experiment.experiment_id)
mlflow.set_experiment("Iris")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1764383013240, experiment_id='2', last_update_time=1764383845253, lifecycle_stage='active', name='Iris', tags={}>

In [ ]:
# Option 2: Permanently delete and recreate
import mlflow
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name("Iris")
client.delete_experiment(experiment.experiment_id)
mlflow.set_experiment("Iris")


In [ ]:
# Option 3: Use a different name
mlflow.set_experiment("Iris-v2")